In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mat4py
!pip install dtcwt
!pip install antropy
!pip install sru

     |████████████████████████████████| 70 kB 3.7 MB/s 
  Created wheel for dtcwt: filename=dtcwt-0.12.0-py3-none-any.whl size=87885 sha256=f8e0cdbdb4477d7752f4ba1cbe8c57414ecedfe13e5dab8d36e4b85123c20b25
  Stored in directory: /root/.cache/pip/wheels/36/17/44/41d78d570172185f944f9b79f983905ab9072d70dbdb118710
Successfully built dtcwt
     |████████████████████████████████| 49 kB 2.8 MB/s 
  Created wheel for antropy: filename=antropy-0.1.4-py3-none-any.whl size=16158 sha256=0197d780beb8f18b10f172face8101020cc4679589873186964c30c298381c65
  Stored in directory: /root/.cache/pip/wheels/d4/cc/6b/28fa7036bdabc6f1f92fd6a108ae259dbb48f107f676d6b491
Successfully built antropy
     |████████████████████████████████| 108 kB 7.7 MB/s 


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mat4py import loadmat
import antropy as ant
from scipy.stats import entropy
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,LSTM
from sru import SRU, SRUCell
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)
from torch.autograd import Variable 
from torch import optim
from matplotlib.pylab import *
import dtcwt
import os

In [ ]:
def training(mav,y_train,y_test,epochs,model_number):
  input_size, hidden_size,num_layers,num_classes = 62, 128,2,3
  if(model_number==1):
    model=Model(62,hidden_size,num_layers,num_classes)
  else:
    model=Model2(62,hidden_size,num_layers,num_classes)
  model.cuda()
  # First 9 sessions
  X_train=mav.iloc[:,:9]
  # last 6 sessions
  X_test=mav.iloc[:,9:15]
  # Reshaping Input as (sample,timestep,features) = (9,19,62)
  train=[]
  a=X_train.T
  for i in range(9):
    for k in range(19):
      timestep=[]
      for j in range(62):
        timestep.append(a.iloc[i][j][k])
      train.append(timestep)
  # Reshaping Input as (sample,timestep,features) = (6,19,62)
  test=[]
  a=X_test.T
  for i in range(6):
    for k in range(19):
      timestep=[]
      for j in range(62):
        timestep.append(a.iloc[i][j][k])
      test.append(timestep)
  train=np.array(train).reshape(19,9,62)
  test=np.array(test).reshape(19,6,62)
  from torch.autograd import Variable 
  X_train_tensors = Variable(torch.tensor(list(train),dtype = torch.cfloat))
  X_test_tensors  = Variable(torch.tensor(list(test),dtype = torch.cfloat))
  y_train_tensors  = torch.tensor(list(y_train))
  y_test_tensors  = torch.tensor(list(y_test)) 
  X_train_tensors=X_train_tensors.to(device)
  X_test_tensors=X_test_tensors.to(device)
  y_train_tensors=y_train_tensors.to(device)
  y_test_tensors=y_test_tensors.to(device)
  from torch import optim
  optimizer = optim.Adam(model.parameters(), lr= 0.0001,weight_decay=1e-5)
  criterion = nn.CrossEntropyLoss()
  train_loss=[]
  test_loss=[]
  from torch.autograd import Variable
  for epoch in range(epochs):
    out=model.forward(X_train_tensors)
    optimizer.zero_grad()
    loss=criterion(out,y_train_tensors.type(torch.int64))
    train_loss.append(loss)
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item())
    val=model.forward(X_test_tensors)
    loss2=criterion(out,y_train_tensors.type(torch.int64))
    test_loss.append(loss2.item())
    print("Epoch" ,epoch+1," loss : ",loss.item())
  pred=model.forward(X_test_tensors)
  _, predicted = torch.max(pred, 1)
  acc = torch.sum(predicted == y_test_tensors)/len(predicted)
  return (train_loss,test_loss,predicted,acc)

In [ ]:
os.listdir('/content/drive/MyDrive/eeg/')

['6_20130712.mat',
 '5_20140418.mat',
 '5_20140411.mat',
 '4_20140705.mat',
 '4_20140702.mat',
 '4_20140621.mat',
 '3_20140629.mat',
 '3_20140611.mat',
 '3_20140603.mat',
 '2_20140419.mat',
 '2_20140413.mat',
 '2_20140404.mat',
 '1_20131107.mat',
 '1_20131030.mat',
 '5_20140506.mat',
 '1_20131027.mat',
 'label.mat',
 'mav_delta.csv']

In [ ]:
file_path=['1_20131027.mat','1_20131030.mat', '1_20131107.mat','1_20131107.mat','4_20140705.mat','4_20140702.mat','4_20140621.mat','3_20140629.mat','3_20140611.mat','3_20140603.mat','2_20140419.mat']
acc_within_one_day=[]
acc_within_one_day2=[]

for file_name in file_path:
  path='/content/drive/MyDrive/eeg/'+ file_name
  data = loadmat(path)
  data=pd.DataFrame(data)
  path=r'/content/drive/MyDrive/eeg/label.mat'
  Trial_labels = pd.DataFrame(loadmat(path))
  data_dtcwt=data.copy()
  data_dtcwt_delta=data.copy()
  data_dtcwt_theta=data.copy()
  data_dtcwt_alpha=data.copy()
  data_dtcwt_beta=data.copy()
  data_dtcwt_gamma=data.copy()
  transform = dtcwt.Transform1d()
  for row in range(data.shape[0]):
      for col in data.columns:
          #checking if size of eeg_data is even
          if(len(data.loc[row,col])%2!=0):
              eeg_data=np.array(data.loc[row,col][:-1])
          else:
              eeg_data=np.array(data.loc[row,col])
          vecs_t = transform.forward(eeg_data, nlevels=5)
          data_dtcwt_delta.loc[row,col]=vecs_t.highpasses[0]
          data_dtcwt_theta.loc[row,col]=vecs_t.highpasses[1]
          data_dtcwt_alpha.loc[row,col]=vecs_t.highpasses[2]
          data_dtcwt_beta.loc[row,col]=vecs_t.highpasses[3]
          data_dtcwt_gamma.loc[row,col]=vecs_t.highpasses[4]
          data_dtcwt.loc[row,col]= transform.inverse(vecs_t)   

  mav_delta=data_dtcwt_delta.copy()
  mav_theta=data_dtcwt_theta.copy()
  mav_alpha=data_dtcwt_alpha.copy()
  mav_beta=data_dtcwt_beta.copy()
  mav_gamma=data_dtcwt_gamma.copy()
  mavs=[mav_delta,mav_theta,mav_alpha,mav_beta,mav_gamma]
  for mav in mavs:
    for row in range(data.shape[0]):
        for col in data.columns:
            eeg_data=mav.loc[row,col]
            eeg_data=list(eeg_data.reshape(1,-1)[0])

            mav_list=[]
            length=int(np.ceil(mav.loc[row,col].shape[0]/19))
            #print(length)
            for i in range(mav.loc[row,col].shape[0]//length+1):
              
              mav_list.append(np.log(np.mean(eeg_data[length*i:length*(i+1)])))
            mav_list=np.array(mav_list).astype('complex128')
            mav_delta.loc[row,col]=mav_list
  data_dtcwt_list=[data_dtcwt_delta,data_dtcwt_theta,data_dtcwt_alpha,data_dtcwt_beta,data_dtcwt_gamma]
  psd_delta=data_dtcwt_delta.copy()
  psd_theta=data_dtcwt_theta.copy()
  psd_alpha=data_dtcwt_alpha.copy()
  psd_beta=data_dtcwt_beta.copy()
  psd_gamma=data_dtcwt_gamma.copy()
  # print(psd_delta.iloc[0,0][0:5])
  psds=[psd_delta,psd_theta,psd_alpha,psd_beta,psd_gamma]
  for (psd,data_) in zip(psds,data_dtcwt_list):
    for row in range(data.shape[0]):
        for col in data.columns:
            #Get the eeg signal 
            sig=data_.loc[row,col]
            #Calculate Power Spectral Density
            PSD=[]
            length=int(np.ceil(psd.loc[row,col].shape[0]/19))
            for i in range(psd.loc[row,col].shape[0]//length+1): 
              psd_batch=np.mean(sig[length*i:length*(i+1)]*np.conj(sig[length*i:length*(i+1)]))/length
              PSD.append(psd_batch) 
            PSD=np.array(PSD).astype('complex128')
            # print(PSD.shape)
            psd.loc[row,col]=PSD
        # psd.shape
  # psd_delta.shape
  import antropy as ant
  dfa_delta=data_dtcwt_delta.copy()
  dfa_theta=data_dtcwt_theta.copy()
  dfa_alpha=data_dtcwt_alpha.copy()
  dfa_beta=data_dtcwt_beta.copy()
  dfa_gamma=data_dtcwt_gamma.copy()
  dfas=[dfa_delta,dfa_theta,dfa_alpha,dfa_beta,dfa_gamma]
  for (dfa,data_) in zip(dfas,data_dtcwt_list):
    for row in range(data.shape[0]):
        for col in data.columns:
            #Get the eeg signal 
            sig=data_.loc[row,col]
            #Calculate Fractal Dimension
            DFA=[]
            length=int(np.ceil(dfa.loc[row,col].shape[0]/19))
            for i in range(dfa.loc[row,col].shape[0]//length+1): 
              batch=np.real(sig[length*i:length*(i+1)])
              batch=np.squeeze(batch)
              dfa_batch=ant.higuchi_fd(batch)
              DFA.append(dfa_batch) 
            DFA=np.array(DFA)
            # print(dfa.shape)
            dfa.loc[row,col]=DFA
  dfa_delta.shape

  # dfa_data=data.copy()
  # for row in range(data.shape[0]):
  #     for col in data.columns:
  #         #Get the eeg signal 
  #         sig=np.array(data.loc[row,col])
  #         #Calculate Higuchi Fractal Dimension 
  #         dfa = hfda.measure(sig, 5)
  #         dfa_data.loc[row,col]=dfa

  from scipy.stats import entropy

  entropy_delta=data_dtcwt_delta.copy()
  entropy_theta=data_dtcwt_theta.copy()
  entropy_alpha=data_dtcwt_alpha.copy()
  entropy_beta=data_dtcwt_beta.copy()
  entropy_gamma=data_dtcwt_gamma.copy()
  # print(entropy_delta.iloc[0,0][0:5])
  entropys=[entropy_delta,entropy_theta,entropy_alpha,entropy_beta,entropy_gamma]
  for (ent,data_) in zip(entropys,data_dtcwt_list):
    for row in range(data.shape[0]):
        for col in data.columns:
            #Get the eeg signal 
            sig=data_.loc[row,col]
            ENTROPY=[]
            length=int(np.ceil(ent.loc[row,col].shape[0]/19))
            for i in range(ent.loc[row,col].shape[0]//length+1): 
              #Get the eeg signal 
              sig_batch=np.real(sig[length*i:length*(i+1)])
              sig_batch=np.squeeze(sig_batch)
              pd_series = pd.Series(sig_batch)
              #Get count of each value for calculating probabilty for entropy 
              counts = pd_series.value_counts()
              #Calculate Entropy
              entropy_batch = entropy(counts) 
              ENTROPY.append(entropy_batch) 
            # print(entropy.shape)
            ENTROPY=np.array(ENTROPY)
            ent.loc[row,col]=ENTROPY
  entropy_delta.shape
  y_train=np.array(Trial_labels['label'][0:9])+1
  y_test=np.array(Trial_labels['label'][9:15])+1

  device = torch.device("cuda")

  class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
      super(Model, self).__init__()
      self.num_layers = num_layers
      self.hidden_size = hidden_size
      self.rnn = SRU(input_size, hidden_size,
                num_layers = 2,          # number of stacking RNN layers
                dropout = 0.0,           # dropout applied between RNN layers
                rnn_dropout = 0.0,       # variational dropout applied on linear transformation
                bidirectional = False,   # bidirectional RNN ?
                layer_norm = False,      # apply layer normalization on the output of each layer
                highway_bias = 0         # initial bias of highway gate (<= 0)
                )
      self.lin=nn.Linear(128,3)
      self.rnn.cuda()
      self.lin.cuda()
          
    def forward(self,x):
      output_states, c_states = self.rnn(x.float())
      # forward pass
      output_states = output_states[-1,:, :]
      out=self.lin(output_states)
      return out

  class Model2(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
      super(Model2, self).__init__()
      self.num_layers = num_layers
      self.hidden_size = hidden_size
      self.rnn1 = SRUCell(input_size=input_size,hidden_size=200)
      self.rnn2= SRUCell(input_size=200,hidden_size=100)
      self.lin1=nn.Linear(100,50)
      self.lin2=nn.Linear(50,3)
      self.drop=nn.Dropout(p=0.5)
      self.rnn1.cuda()
      self.rnn2.cuda()
      self.lin1.cuda()
      self.lin2.cuda()
      
          
    def forward(self,x):
      output_states, c_states = self.rnn1(x.float())
      output_states, c_states = self.rnn2(output_states)
      # forward pass
      output_states = output_states[-1,:, :]
      output_states=self.lin1(output_states)
      output_states=self.drop(output_states)
      out=self.lin2(output_states)
      return out

  result_mav_delta=training(mav_delta,y_train,y_test,200,1)
  result_mav_theta=training(mav_theta,y_train,y_test,200,1) 
  result_mav_alpha=training(mav_alpha,y_train,y_test,200,1)
  result_mav_beta=training(mav_beta,y_train,y_test,200,1)
  result_mav_gamma=training(mav_gamma,y_train,y_test,200,1)

  result_psd_delta=training(psd_delta,y_train,y_test,200,2)
  result_psd_theta=training(psd_theta,y_train,y_test,200,2)
  result_psd_alpha=training(psd_alpha,y_train,y_test,200,2)
  result_psd_beta=training(mav_beta,y_train,y_test,200,2)
  result_psd_gamma=training(mav_gamma,y_train,y_test,100,2)

  result_dfa_delta=training(dfa_delta,y_train,y_test,100,2)
  result_dfa_theta=training(dfa_theta,y_train,y_test,50,2)
  result_dfa_alpha=training(dfa_alpha,y_train,y_test,50,2)
  result_dfa_beta=training(dfa_beta,y_train,y_test,50,2)
  result_dfa_gamma=training(dfa_gamma,y_train,y_test,50,2)

  result_entropy_delta=training(entropy_delta,y_train,y_test,100,2)
  result_entropy_theta=training(entropy_theta,y_train,y_test,50,2)
  result_entropy_alpha=training(entropy_alpha,y_train,y_test,100,2)
  result_entropy_beta=training(entropy_beta,y_train,y_test,100,2)
  result_entropy_gamma=training(entropy_gamma,y_train,y_test,100,2)

  pred=[]
  max_accuracy=-1*float("inf")
  for x in [result_mav_delta,result_psd_delta,result_dfa_delta,result_entropy_delta]:
    if(x[3]>max_accuracy):
      max_accuracy=x[3]
      max_i=x[2]
  pred.append(max_i)

  max_accuracy=-1*float("inf")
  for x in [result_mav_theta,result_psd_theta,result_dfa_theta,result_entropy_theta]:
    if(x[3]>max_accuracy):
      max_accuracy=x[3]
      max_i=x[2]
  pred.append(max_i)

  max_accuracy=-1*float("inf")
  for x in [result_mav_alpha,result_psd_alpha,result_dfa_alpha,result_entropy_alpha]:
    if(x[3]>max_accuracy):
      max_accuracy=x[3]
      max_i=x[2]
  pred.append(max_i)

  max_accuracy=-1*float("inf")
  for x in [result_mav_beta,result_psd_beta,result_dfa_beta,result_entropy_beta]:
    if(x[3]>max_accuracy):
      max_accuracy=x[3]
      max_i=x[2]
  pred.append(max_i)

  max_accuracy=-1*float("inf")
  for x in [result_mav_gamma,result_psd_gamma,result_dfa_gamma,result_entropy_gamma]:
    if(x[3]>max_accuracy):
      max_accuracy=x[3]
      max_i=x[2]
  pred.append(max_i)
  prediction=[]
  for i in range(6):
    pos=0
    neg=0
    neu=0
    for p in pred:
      if(p[i]==0):
        neg+=1
      elif(p[i]==1):
        neu+=1
      else:
        pos+=1
    max_emotion=max(pos,max(neg,neu))
    print(neg," ",neu," ",pos)
    if(neg==max_emotion):
      prediction.append(0)
    elif(pos==max_emotion):
      prediction.append(2)
    else:
      prediction.append(1)

  
  prediction = torch.tensor(list(prediction))
  prediction=prediction.to(device)
  y_test_tensors  = torch.tensor(list(y_test))
  y_test_tensors=y_test_tensors.to(device)
  acc = torch.sum(prediction == y_test_tensors)/len(prediction)
  acc_within_one_day.append(acc)
  print('Accuracy with strategy 1 : ',acc)
  
  prediction2=[]
  for i in range(6):
    pos=0
    neg=0
    neu=0
    for p in [result_mav_gamma,result_psd_gamma,result_dfa_gamma,result_entropy_gamma,result_mav_beta,result_psd_beta,result_dfa_beta,result_entropy_beta]:
      #print(p)
      if(p[2][i]==0):
        neg+=1
      elif(p[2][i]==1):
        neu+=1
      else:
        pos+=1
    max_emotion=max(pos,max(neg,neu))
    print(neg," ",neu," ",pos)
    if(neg==max_emotion):
      prediction2.append(0)
    elif(pos==max_emotion):
      prediction2.append(2)
    else:
      prediction2.append(1)
  prediction2 = torch.tensor(list(prediction2))
  prediction2=prediction2.to(device)
  y_test_tensors  = torch.tensor(list(y_test))
  y_test_tensors=y_test_tensors.to(device)
  acc = torch.sum(prediction2 == y_test_tensors)/len(prediction2)
  acc_within_one_day2.append(acc)
  print('Accuracy with strategy 2 : ',acc)
  

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in cdouble_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:164: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  ../aten/src/ATen/native/Copy.cpp:244.)


Streaming output truncated to the last 5000 lines.
Epoch 129  loss :  0.0015448142075911164
Epoch 130  loss :  0.001534836832433939
Epoch 131  loss :  0.0015249785501509905
Epoch 132  loss :  0.0015151991974562407
Epoch 133  loss :  0.001505538821220398
Epoch 134  loss :  0.0014959973050281405
Epoch 135  loss :  0.0014865612611174583
Epoch 136  loss :  0.0014772046124562621
Epoch 137  loss :  0.0014679530868306756
Epoch 138  loss :  0.0014587946934625506
Epoch 139  loss :  0.0014497548108920455
Epoch 140  loss :  0.0014407939743250608
Epoch 141  loss :  0.0014318861067295074
Epoch 142  loss :  0.0014231367968022823
Epoch 143  loss :  0.0014144269516691566
Epoch 144  loss :  0.0014058755477890372
Epoch 145  loss :  0.0013973771128803492
Epoch 146  loss :  0.0013889444526284933
Epoch 147  loss :  0.0013806043425574899
Epoch 148  loss :  0.0013723435113206506
Epoch 149  loss :  0.0013641883851960301
Epoch 150  loss :  0.001356072723865509
Epoch 151  loss :  0.0013480494963005185
Epoch 152

Accuracy calculation

In [ ]:
acc_within_one_day

[tensor(0.8333, device='cuda:0'),
 tensor(0.8333, device='cuda:0'),
 tensor(0.5000, device='cuda:0'),
 tensor(0.8333, device='cuda:0'),
 tensor(0.6667, device='cuda:0'),
 tensor(0.5000, device='cuda:0'),
 tensor(0.5000, device='cuda:0'),
 tensor(0.5000, device='cuda:0'),
 tensor(0.6667, device='cuda:0'),
 tensor(1., device='cuda:0'),
 tensor(0.3333, device='cuda:0')]

In [ ]:
# mean of Strategy 1 ensemble(best from each frequency band)
acc_within_one_day=torch.Tensor(acc_within_one_day).to(device)
torch.mean(acc_within_one_day)

tensor(0.6515, device='cuda:0')

In [ ]:
acc_within_one_day2

[tensor(0.6667, device='cuda:0'),
 tensor(0.5000, device='cuda:0'),
 tensor(0.1667, device='cuda:0'),
 tensor(0.3333, device='cuda:0'),
 tensor(0.1667, device='cuda:0'),
 tensor(0.3333, device='cuda:0'),
 tensor(0.1667, device='cuda:0'),
 tensor(0.5000, device='cuda:0'),
 tensor(0.1667, device='cuda:0'),
 tensor(0.3333, device='cuda:0'),
 tensor(0.1667, device='cuda:0')]

In [ ]:
# mean of Strategy 2(2 Highest frequency bands- gamma and beta)
acc_within_one_day2=torch.Tensor(acc_within_one_day2).to(device)
torch.mean(acc_within_one_day2)

tensor(0.3182, device='cuda:0')

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test,prediction.cpu().detach().numpy(), labels=[0,1,2])
import seaborn as sns

ax = sns.heatmap(matrix, annot=True, cmap='Blues')

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['Negative','Neutral','Positive'])
ax.yaxis.set_ticklabels(['Negative','Neutral','Positive'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test,prediction.cpu().detach().numpy(),average='micro')